In [2]:
import numpy as np
import pandas as pd

- `groupby()`: group data by column, array, series, function, dictionary, index level; compute stuff on resulting groups
- groups can be iterated simply by doing `for name, group in df.groupby(...)`
- many built-in functions exist for computing aggregations: e.g. `mean`, `std`, `min`, `count`...
- `.agg()` use used to apply a custom aggregation function (or dictionary)
    - Purpose: gives summary statistics per group, aggregation/reduction
    - Output shape: one row per group
    - E.g. for mean, sum, counts...
- `apply()` is a general purpose groupby method: it works by splitting the manipulated object into pieces, invoking some function to each piece on its own and concatenating the result
    - Purpose: most general purpose, do anything you want
    - Output shape: Anything
- `transform()` is a method for grouped operations, where the result will have the same index as the object that was split into groups
    - Purpose: Group-wise transformation, compute something per group and broadcast it back
    - Output shape: Same length as original data
- `pivot_table()` is a function that aggregates data into a table, with group keys along the resulting columns and rows
- `crosstab()` is a special pivot table that simply computes group frequencies

# 10.1 How to Think About Group Operations

In [3]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-0.952613,-0.566805
1,a,2,1.440011,-0.127155
2,None,1,-0.313519,1.944650
3,b,2,-0.241697,-0.714021
4,b,1,-0.914003,0.151132
5,a,<NA>,-0.127244,1.893940
6,None,1,0.478733,-1.499507


In [4]:
# one way of grouping data
# by calling groupby(df["key1"]), data has been aggregated by splitting the data into groups
# producing a new series that has the unique values in key1 as an index
grouped = df["data1"].groupby(df["key1"])
grouped

In [5]:
grouped.mean()

key1
a    0.083039
b    0.557149
Name: data1, dtype: float64

In [9]:
# using multiple keys, we get a hierarchical index
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1       0.759640
      2      -0.232260
b     1       0.928681
      2       0.185617
Name: data1, dtype: float64

In [10]:
means.unstack()

key2,1,2
key1,,
a,0.759640,-0.232260
b,0.928681,0.185617


In [11]:
# the keys can be any array of the right length
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]
df["data1"].groupby([states, years]).mean()

CA  2005   -0.255262
    2006    0.734372
OH  2005    0.472629
    2006   -0.010788
Name: data1, dtype: float64

In [12]:
# you can also pass column names if the grouping information is found in the same dataframe
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,0.083039,-0.174907
b,1.5,0.557149,0.596174


In [13]:
# since key1 is not numeric, we have to pass in numeric_only!
df.groupby("key2").mean(numeric_only=True)

,data1,data2
key2,,
1,0.368109,0.309393
2,-0.023321,0.390289


In [14]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1     0.759640 -1.055917
     2    -0.232260  1.270812
b    1     0.928681  1.682581
     2     0.185617 -0.490234

In [15]:
# another useful operation is size()
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [16]:
# any missing keys are excluded by default, which can be overwritten
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [18]:
# same per column
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


- create a group by calling `df.groupby(key)`
- `key` can take on many forms: it could be an array, a series, a column identifier
- key can contain multiple elements, as well
- the groups can be used to calculate summary statistics:
    - `mean()`: calculate the mean of each column per group, pass in `numeric_only` to exclude non-numeric columns
    - `size()`: count the number of elements per group
    - `count()`: count the number of distinct elements in each column per group
- use the parameter `dropna` in `groupby()` to control whether not available elements are ignored or not

## Iterating over Groups

In [19]:
# the object can be iterated
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1  0.759640 -1.055917
1    a     2 -0.232260  1.270812
5    a  <NA> -0.278264 -0.739616
b
  key1  key2     data1     data2
3    b     2  0.185617 -0.490234
4    b     1  0.928681  1.682581


In [20]:
# when using multiple keys, the first returned object will have multiple keys
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', np.int64(1))
  key1  key2    data1     data2
0    a     1  0.75964 -1.055917
('a', np.int64(2))
  key1  key2    data1     data2
1    a     2 -0.23226  1.270812
('b', np.int64(1))
  key1  key2     data1     data2
4    b     1  0.928681  1.682581
('b', np.int64(2))
  key1  key2     data1     data2
3    b     2  0.185617 -0.490234


In [21]:
# we can also group by columns
# this is deprecated now!
grouped = df.T.groupby({"key1": "key", "key2": "key", "data1": "data", "data2": "data"})
grouped

/tmp/ipykernel_10671/1253761969.py:2: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df.groupby({"key1": "key", "key2": "key", "data1": "data", "data2": "data"}, axis="columns")


- `df.groupby()` returns a 2-tuple, which can be iterated: `for name, group in df.groupby("key1"):`
- if several keys are used, the first element is a tuple itself: `for (k1, k2), group in df.groupby(["key1", "key2"]):`
- we can group by columns as well:
    - the book mentions to use `df.groupby("key1", axis="columns")`
    - this is deprecated!
    - use `df.T.groupby("key1")` instead

## Selecting a Column or Subset of Columns

In [22]:
# this
print(df.groupby("key1")["data1"])
df.groupby("key1")[["data2"]]

In [ ]:
# is the same as this
print(df["data1"].groupby(df["key1"]))
df[["data2"]].groupby(df["key1"])

In [23]:
# e.g. to compute the mean of just one column
df.groupby("key1")["data2"].mean()

key1
a   -0.174907
b    0.596174
Name: data2, dtype: float64

- to only get certain columns from the group, we can use column indexing: `df.groupby(["key1", "key2"])[["data2"]]`
- the returned object is either `pd.Series` or `pd.DataFrame`

## Grouping with Dictionaries and Series

In [26]:
people = pd.DataFrame(np.random.standard_normal((5, 5)), columns=["a", "b", "c", "d", "e"], index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,0.215557,1.037784,-0.089122,-0.677625,0.362454
Steve,-1.827740,0.753775,0.883362,-0.600481,1.329413
Wanda,1.721957,NaN,NaN,1.744240,-0.080745
Jill,1.266912,-1.208481,0.289424,-0.768405,-0.846620
Trey,0.386572,-0.162490,-0.786491,-1.379098,1.229654


In [41]:
# we can also use dictionaries to group the data
mapping = {"a": "red", "b": "red", "c": "blue", "d": "blue", "e": "red", "f": "orange"}
by_column = people.T.groupby(mapping)
by_column.sum().T

,blue,red
Joe,-0.766747,1.615795
Steve,0.282881,0.255448
Wanda,1.744240,1.641211
Jill,-0.478981,-0.788189
Trey,-2.165590,1.453736


In [42]:
# or even series
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [43]:
people.T.groupby(map_series).count().T

,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


- you can use a dictionary to map keys to groups
- and `pd.Series`, too
- simply by passing to the function: `df.groupby(dict)` or `df.groupby(series)`

## Grouping with Functions

In [44]:
# the function passed as a key will be called once per index value
people.groupby(len).sum()

,a,b,c,d,e
3,0.215557,1.037784,-0.089122,-0.677625,0.362454
4,1.653484,-1.370971,-0.497067,-2.147503,0.383034
5,-0.105783,0.753775,0.883362,1.143760,1.248667


In [45]:
# we can even mix functions with other key types
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,0.215557,1.037784,-0.089122,-0.677625,0.362454
4,two,0.386572,-1.208481,-0.786491,-1.379098,-0.846620
5,one,-1.827740,0.753775,0.883362,-0.600481,-0.080745


- a function can be used to group items: `df.groupby(len)`
- the function will be called on the index
- functions can be combined with other keys as well

## Grouping by Index Levels

In [47]:
columns = pd.MultiIndex.from_arrays([
    ["US", "US", "US", "JP", "JP"],
    [1, 3, 5, 1, 3]],
    names=["cty", "tenor"])

hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.668758  0.893425  0.402651 -0.458339  0.402515
1      2.122335  0.548237 -0.082681  0.376809 -0.434792
2     -0.690101 -0.277649  0.565470  0.042014  0.557766
3      1.087274 -0.284234  0.771230 -0.879276  1.497131

In [50]:
# we can also group by certain level of a hierarchical index
hier_df.T.groupby(level="cty").count().T

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


- for hierarchical datasets, we can group items by index and pass in the index level: `df.groupby(level="level")`

# 10.2 Data Aggregation

In [4]:
# many different optimized methods for the groups exist (check below for overview)
# and also methods of the objects the group consists of can be used
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)

key1   
a     0   -0.952613
      5   -0.127244
b     4   -0.914003
      3   -0.241697
Name: data1, dtype: float64

In [5]:
# you can create custom grouping functions too, passed into agg()
def peak_to_peak(arr):
    """Computes max - min"""
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,2.392624,2.460746
b,1,0.672306,0.865153


In [6]:
# describe is also defined on group-level
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.120051  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0 -0.577850  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.656383  1.440011   3.0  0.399993  1.312338 -0.566805 -0.346980   
b    -0.409773 -0.241697   2.0 -0.281445  0.611755 -0.714021 -0.497733   

                                    
           50%       75%       max  
key1                                
a    -0.127155  0.883392  1.893940  
b    -0.281445 -0.065156  0.151132  

[2 rows x 24 columns]

- Aggregations are data transformations that produce scalar values from arrays, there are many examples:
    - `any, all`: Return `True` if any or all non-NA values are "truthy"
    - `count`: Number of non-NA values
    - `mean`: Mean of non-NA values
    - `median`: Arithmetic mean of non-NA values
    - `min, max`: Mean and max of non-NA values
    - `nth`: Retrieve value that would appear at position `n` in sorted order
    - `ohlc`: Compute "open-high-low-close" statistics for time-series data
    - `prod`: Product
    - `quantile`: Compute sample quantile
    - `size`: Compute group size
    - `sum`: Compute sum of non-NA values
    - `std, var`: Compute standard deviation and variance
- You can create your own aggregations by defining a function that takes in the array and returns a scalar `def fun(arr) -> int` and calling `grouped.agg(fun)`
- `describe()` is defined on the function as well

## Column-Wise and Multiple Function Application

In [37]:
tips = pd.read_csv("examples/tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [38]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [9]:
grouped = tips.groupby(["day", "smoker"])
grouped_pct = grouped["tip_pct"]

# functions can be passed in by name, too
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [10]:
# a list can be used, as well
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [12]:
# you can pass in tuples to define names for the new columns
grouped_pct.agg([("average", "mean"), ("stdev", "std")])

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [13]:
functions = ["count", "mean", "max"]

# when picking certain columns, the functions will be called only on those
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [14]:
# the result has a hierarchical index
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [15]:
# a list of tuples with custom names can be used again
ftuples = [("Average", "mean"), ("Variance", "var")]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [17]:
# if you want to compute different functions per column, use a dictionary:
grouped.agg({"tip": "max", "size": "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [18]:
# the values can be a list, too
grouped.agg({"tip_pct": ["min", "max", "mean", "std"], "size": "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

- computing a statistic on a single dataframe column will return a series
- you can pass in a list to  `df.agg([])` to compute several statistics on the columns
- for convenience, the functions can be defined as tuples with 2 elements: a name and the function itself
- the name will be used as a column identifier
- picking certain columns from the df and calling `.agg()` will create a hierarchical index with the aggregation functions on the lowest level
- you can pass in dict to `df.agg({})`, too: in this case, the keys are columns and the values are a function or lists of functions

## Returning Aggregated Data Without Row Indexes

In [20]:
# if you do not want the data to be returned with an index, you can disable this
grouped = tips.groupby(["day", "smoker"], as_index=False)
grouped.mean(numeric_only=True)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


- pass in `as_index` to `df.groupby()` to return the group with a reset index
- in this case, the group identifiers are added as columns

# 10.3 Apply: General split-apply-combine

`apply` is a group method that splits the object being manipulated into multiple pieces, invokes some function on each piece and then concatenates the results.

In [54]:
def top(dat, n=5, column="tip_pct"):
    """Returns top n values"""
    return dat.sort_values(column, ascending=False)[:n]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [36]:
# now we can use this function using apply:
# essentially, tips is grouped based on the value of smoker
# then the function is called on each group
# and then the result is concatenated into a new dataframe
tips.groupby("smoker").apply(top, include_groups=False)

total_bill   tip   day    time  size   tip_pct
smoker                                                    
No     232       11.61  3.39   Sat  Dinner     2  0.291990
       149        7.51  2.00  Thur   Lunch     2  0.266312
       51        10.29  2.60   Sun  Dinner     2  0.252672
       185       20.69  5.00   Sun  Dinner     5  0.241663
       88        24.71  5.85  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15   Sun  Dinner     2  0.710345
       178        9.60  4.00   Sun  Dinner     2  0.416667
       67         3.07  1.00   Sat  Dinner     1  0.325733
       183       23.17  6.50   Sun  Dinner     4  0.280535
       109       14.31  4.00   Sat  Dinner     2  0.279525

In [38]:
# describe is a good example for apply
# internally, pandas simply calls grouped.apply(describe)
result = tips.groupby("smoker")["tip_pct"].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


- `apply` is a general-purpose method for performing computations on each group
- it splits the data, manipulates it and concatenates the pieces together again with a hierarchical index
- the function takes in another function to execute: `df.groupby("key").apply(func)`
- any arguments to `func` are simply passed to apply: `apply(func, kw_arg1 = value)`
- whatever happens inside func does not matter, but it must return a scalar value or a pandas object

## Suppressing the Group Keys

In [39]:
# the book mentions here that we can use group_keys=False to create a returned dataframe without a hierarchical index
tips.groupby("smoker", group_keys=False).apply(top)

/tmp/ipykernel_18469/3851957478.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby("smoker", group_keys=False).apply(top)


,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [57]:
# as the output suggests, this is deprecated now!
# we can get the same result with a bit of magic
tips.groupby("smoker").apply(top, include_groups=False).reset_index().set_index("level_1")

,smoker,total_bill,tip,day,time,size,tip_pct
level_1,,,,,,,
232,No,11.61,3.39,Sat,Dinner,2,0.291990
149,No,7.51,2.00,Thur,Lunch,2,0.266312
51,No,10.29,2.60,Sun,Dinner,2,0.252672
185,No,20.69,5.00,Sun,Dinner,5,0.241663
88,No,24.71,5.85,Thur,Lunch,2,0.236746
172,Yes,7.25,5.15,Sun,Dinner,2,0.710345
178,Yes,9.60,4.00,Sun,Dinner,2,0.416667
67,Yes,3.07,1.00,Sat,Dinner,1,0.325733
183,Yes,23.17,6.50,Sun,Dinner,4,0.280535


- if we do not want to have an object with a hierarchical index, use `include_groups=False` together with `reset_index()` and `set_index()`

## Quantile and Bucket Analysis

In [5]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000), "data2": np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,-2.513650,1.808655
1,0.043248,0.563524
2,1.009780,0.389349
3,-0.045919,-1.388928
4,0.860875,2.176134


In [6]:
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(10)

0    (-2.913, -1.384]
1     (-1.384, 0.138]
2      (0.138, 1.661]
3     (-1.384, 0.138]
4      (0.138, 1.661]
5     (-1.384, 0.138]
6     (-1.384, 0.138]
7      (1.661, 3.183]
8      (0.138, 1.661]
9      (0.138, 1.661]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-2.913, -1.384] < (-1.384, 0.138] < (0.138, 1.661] < (1.661, 3.183]]

In [7]:
# we can pass the Categorical from pandas.cut directly to groupby
def get_stats(group_f):
    """Calculates some statistics on the group"""
    return pd.DataFrame(
        {"min": group_f.min(), "max": group_f.max(), "count": group_f.count(), "mean": group_f.mean()}
    )

grouped = frame.groupby(quartiles, observed=True)
grouped.apply(get_stats)

min       max  count      mean
data1                                                      
(-2.913, -1.384] data1 -2.906549 -1.387405     73 -1.853724
                 data2 -2.422497  2.338773     73 -0.111184
(-1.384, 0.138]  data1 -1.379268  0.134455    470 -0.506393
                 data2 -2.950829  3.563776    470 -0.021249
(0.138, 1.661]   data1  0.138551  1.643549    415  0.744397
                 data2 -2.895345  2.642790    415  0.009249
(1.661, 3.183]   data1  1.661600  3.183172     42  2.080286
                 data2 -1.861639  2.113483     42 -0.091629

In [8]:
# the same result as above could also be calculated like so:
grouped.agg(["min", "max", "count", "mean"])

data1                               data2            \
                       min       max count      mean       min       max   
data1                                                                      
(-2.913, -1.384] -2.906549 -1.387405    73 -1.853724 -2.422497  2.338773   
(-1.384, 0.138]  -1.379268  0.134455   470 -0.506393 -2.950829  3.563776   
(0.138, 1.661]    0.138551  1.643549   415  0.744397 -2.895345  2.642790   
(1.661, 3.183]    1.661600  3.183172    42  2.080286 -1.861639  2.113483   

                                  
                 count      mean  
data1                             
(-2.913, -1.384]    73 -0.111184  
(-1.384, 0.138]    470 -0.021249  
(0.138, 1.661]     415  0.009249  
(1.661, 3.183]      42 -0.091629

In [13]:
# same can be computed on quartiles of equal length, too
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp.head()

0    0
1    1
2    3
3    1
4    3
Name: data1, dtype: int64

In [14]:
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -2.906549 -0.643629    250 -1.237947
      data2 -2.950829  3.563776    250 -0.054814
1     data1 -0.641370  0.043248    250 -0.270814
      data2 -2.148027  3.191859    250 -0.001442
2     data1  0.045958  0.692093    250  0.343633
      data2 -2.895345  2.341420    250 -0.064149
3     data1  0.695298  3.183172    250  1.257008
      data2 -2.600301  2.642790    250  0.047951

- the `Categorical` object returned by `pd.cut()` or `pd.qcut()` can be passed to `groupby()`

## Examples: Filling Missing Values with Group-Specific Values

In [15]:
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s

0         NaN
1   -0.368437
2         NaN
3    1.605443
4         NaN
5    0.566916
dtype: float64

In [16]:
# a common way to deal with missing values is by filling in, e.g. using the mean
s.fillna(s.mean())

0    0.601307
1   -0.368437
2    0.601307
3    1.605443
4    0.601307
5    0.566916
dtype: float64

In [17]:
states = ["Ohio", "New York", "Vermont", "Florida", "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East", "West", "West", "West", "West"]
data = pd.Series(np.random.standard_normal(8), index=states)
data

Ohio          2.244405
New York      1.220145
Vermont       0.552525
Florida      -1.332234
Oregon        1.122844
Nevada        1.297799
California    0.621831
Idaho         0.322567
dtype: float64

In [19]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio          2.244405
New York      1.220145
Vermont            NaN
Florida      -1.332234
Oregon        1.122844
Nevada             NaN
California    0.621831
Idaho              NaN
dtype: float64

In [20]:
data.groupby(group_key).size()

East    4
West    4
dtype: int64

In [21]:
data.groupby(group_key).count()

East    3
West    2
dtype: int64

In [22]:
data.groupby(group_key).mean()

East    0.710772
West    0.872338
dtype: float64

In [23]:
# suppose you want to fill in missing data on a group level
# you can simply group the data and use apply with a function that calls fillna on each data chunk
def fill_mean(group_f):
    """Fills empty values with the group mean"""
    return group_f.fillna(group_f.mean())

data.groupby(group_key).apply(fill_mean)

East  Ohio          2.244405
      New York      1.220145
      Vermont       0.710772
      Florida      -1.332234
West  Oregon        1.122844
      Nevada        0.872338
      California    0.621831
      Idaho         0.872338
dtype: float64

In [24]:
# you can also use a dictionary to fill with predefined values
fill_values = {"East": 0.5, "West": -1}
def fill_func(group_f):
    """Fill based on dictionary"""
    return group_f.fillna(fill_values[group_f.name])

data.groupby(group_key).apply(fill_func)

East  Ohio          2.244405
      New York      1.220145
      Vermont       0.500000
      Florida      -1.332234
West  Oregon        1.122844
      Nevada       -1.000000
      California    0.621831
      Idaho        -1.000000
dtype: float64

- we can fill in missing values per group through `apply(func)`, e.g. by letting the function return a single value
- or by using a dictionary that maps groups to values

## Example: Random Sampling and Permutation

In [3]:
suits = ["H", "S", "C", "D"]
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)
deck.head(13)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [4]:
def draw(deck_f, n=5):
    """Randomly draw 5 samples"""
    return deck_f.sample(n)

draw(deck)

QD    10
8D     8
AH     1
3S     3
2D     2
dtype: int64

In [5]:
# we could now get random cards per suit
def get_suit(card):
    """Returns the last letter of the input"""
    return card[-1]

deck.groupby(get_suit).apply(draw, n=2) # parameters to the function being applied can be passed to apply() directly

C  JC     10
   10C    10
D  KD     10
   QD     10
H  QH     10
   8H      8
S  KS     10
   9S      9
dtype: int64

In [6]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

AC      1
4C      4
10D    10
5D      5
7H      7
JH     10
QS     10
10S    10
dtype: int64

- apply can also be used to sample values per group

## Example: Group Weighted Average and Correlation

In [7]:
df = pd.DataFrame({"category": ["a", "a", "a", "a", "b", "b", "b", "b"], "data": np.random.standard_normal(8), "weights": np.random.uniform(size=8)})
df

,category,data,weights
0,a,-0.245953,0.167030
1,a,-0.610372,0.818542
2,a,0.408519,0.877509
3,a,1.168942,0.989557
4,b,-0.325233,0.970278
5,b,0.941765,0.426775
6,b,-0.759036,0.337512
7,b,-1.620623,0.147291


In [10]:
# using apply, operations between columns of a dataframe can be computed, too
grouped = df.groupby("category")

def get_wavg(group_f):
    """Computes the weighted average."""
    return np.average(group_f["data"], weights=group_f["weights"])

grouped.apply(get_wavg, include_groups=False)

category
a    0.341619
b   -0.217090
dtype: float64

In [11]:
close_px = pd.read_csv("examples/stock_px.csv", parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [12]:
close_px.tail(4)

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [15]:
def spx_corr(group_f):
    """Computes the pairwise correlation with the spx column"""
    return group_f.corrwith(group_f["SPX"])

rets = close_px.pct_change().dropna()

In [16]:
def get_year(x):
    """Returns year"""
    return x.year

by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [18]:
# the same process can be applied to compute intercolumn correlations
def corr_aapl_msft(group_f):
    """Computes intercolumn correlation"""
    return group_f["AAPL"].corr(group_f["MSFT"])

by_year.apply(corr_aapl_msft)

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

- `apply()` also allows computations between columns

## Example: Group-Wise Linear Regression

In [19]:
%conda install statsmodels

2 channel Terms of Service accepted
Retrieving notices: done
Channels:
 - defaults
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.5.1
    latest version: 25.11.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/nico/miniconda3/envs/data-science

  added / updated specs:
    - statsmodels


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    patsy-1.0.1                |  py312h06a4308_0         356 KB
    statsmodels-0.14.6         |  py312h47b2149_0        11.4 MB
    ------------------------------------------------------------
                                           Total:        11.7 MB

The following NEW packages will be INSTALLED:

  patsy              pkgs/main/linux-64::patsy-1.0.1-py312h06a4308_0 
  statsmodels        pkgs/main/

In [21]:
# you can use groupby also to perform more complex group-wise statistical analysis, as long as the function returns a pandas object or scalar value
import statsmodels.api as sm

def regress(_data, yvar=None, xvars=None):
    """Computes a OLS"""
    y = _data[yvar]
    x = _data[xvars]
    x["intercept"] = 1.
    _result = sm.OLS(y, x).fit()
    return _result.params

In [22]:
by_year.apply(regress, yvar="AAPL", xvars=["SPX"])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


- whatever happens in `apply()` does not matter, as long as a scalar value or pandas object is returned

# 10.4 Group Transforms and "Unwrapped" GroupBys

`transform` is similar to `apply`, but with more constraints. The functions it can take in must:
- produce a scalar value to be broadcast to the shape of the group.
- produce an object of the same shape as the input group.
- not mutate the input.

Essentially, a transformation is a group operation whose result is indexed the same as the one being grouped.

In [25]:
df = pd.DataFrame({"key": ["a", "b", "c"] * 4, "value": np.arange(12)})
df

,key,value
0,a,0
1,b,1
2,c,2
3,a,3
4,b,4
5,c,5
6,a,6
7,b,7
8,c,8
9,a,9


In [26]:
g = df.groupby("key")["value"]
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [28]:
# instead of only returning the mean per category, we get a whole series with an index-aligned result
g.transform("mean")

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [29]:
# passing in a function is similar to apply:
def get_mean(group_f):
    """Compute mean"""
    return group_f.mean()

g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [30]:
# transform can be used with functions that return series as well
def times_two(group_f):
    """Returns a group times two"""
    return group_f * 2

g.transform(times_two)

0      0
1      2
2      4
3      6
4      8
5     10
6     12
7     14
8     16
9     18
10    20
11    22
Name: value, dtype: int64

In [31]:
# or compute group-wise ranks
def get_ranks(group_f):
    """Returns rank per group"""
    return group_f.rank(ascending=False)

g.transform(get_ranks)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [32]:
# one possible use case is normalizing on a group level
def normalize(x):
    """Normalize"""
    return (x - x.mean()) / x.std()

g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [33]:
# in this example, we can use apply too
g.apply(normalize)

key    
a    0    -1.161895
     3    -0.387298
     6     0.387298
     9     1.161895
b    1    -1.161895
     4    -0.387298
     7     0.387298
     10    1.161895
c    2    -1.161895
     5    -0.387298
     8     0.387298
     11    1.161895
Name: value, dtype: float64

In [34]:
# built-in aggregate functions are much faster than a general apply
# by combining them with transform, we can do unwrapped group operations:
g.transform("mean")

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [35]:
# unwrapping means that we are doing multiple groupby operations instead of writing a single function to be passed to apply()
# usually this is faster due to vectorization
normalized = (df["value"] - g.transform("mean")) / g.transform("std")
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

- transform is similar to apply, but it returns a series/df of equal size
- this can be leveraged for unwrapped grouping operations, such as `(df["value"] - g.transform("mean")) / g.transform("std")`
- which is usually faster due to vectorization

# 10.5 Pivot Tables and Cross-Tabulation

Pivot tables are data summarization tools that aggregate a table of data by one or more keys and arranging the resulting data in a rectangle with some
group keys along the rows and some other along the columns. Pivot tables can be constructed using `groupby`, `df.pivot_table` or `pd.pivot_table`.

In [39]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [40]:
# the default aggregation type for pivot_table is the mean
tips.pivot_table(index=["day", "smoker"], values=["size", "tip", "tip_pct", "total_bill"])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [42]:
# the same result with groupby:
tips.groupby(["day", "smoker"]).mean(numeric_only=True)

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [43]:
# we can provide further groupings using the columns parameter
tips.pivot_table(index=["time", "day"], columns="smoker", values=["tip_pct", "size"])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [44]:
# by passing in margins=True, we compute further group statistics within a single tier
tips.pivot_table(index=["time", "day"], columns="smoker", values=["tip_pct", "size"], margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [46]:
# we can use other aggregate functions as well
tips.pivot_table(index=["time", "smoker"], columns="day", values="tip_pct", aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [47]:
# a fill value can be specified too
tips.pivot_table(index=["time", "size", "smoker"], columns="day", values="tip_pct", fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

- a pivot table is a summarization tool that aggregates a table of data by one or more keys, while arranging the results based on the group keys
- some group keys will be along the rows, others along the columns
- in `pandas`, this can be achieved either using `groupby()` or by directly using `pd.pivot_table()`/`df.pivot_table()`
- the function takes in:
    - `index`: a list of column identifiers, which will be along the index of the resulting pivot table
    - `columns`: a list of column identifiers, which will be along the column of the resulting pivot table
    - `values`: the values to use for calculation
    - `margins`: whether to compute partial totals
    - `aggfunc`: a different aggregation function
    - `fillna`: a fill value
- the default aggregation function is `mean`

## Cross-Tabulations: Crosstab

A cross-tabulation is a pivot table that computes group frequencies.

In [50]:
from io import StringIO

data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""

data = pd.read_table(StringIO(data), sep="\\s+")
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [51]:
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [52]:
# the first two arguments can be an array, Series or a list of arrays
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

- cross-tabulations are a form of pivot table that computes group frequencies
- pandas provides the built-in function `pd.crosstab`